In [18]:
#generate data do compare network quality
import sys
sys.path.insert(0, '../../helperScripts/')
from helperFunctions import *
import scmra
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
from datetime import datetime
import pickle
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
import scmra

def calculate_diff_dict():
   dict = {}
   dict['braf'] = ((rloc_true - rloc_braf)**2)**0.5
   dict['ras'] = ((rloc_true - rloc_ras)**2)**0.5

   return(dict)

def print_rloc(rlocDict, key, fileLocation):
  rloc = rlocDict.rloc[key]
  rloc = reorder_rloc(rloc)
  rloc.to_csv(fileLocation, sep = "\t")

def calculate_rmse_of_network_differences(rlocList, populationStringList, diffDict):

   rloc_wt = None
   rloc_mut = None

   for popIdx in range(len(populationStringList)):
      pop = populationStringList[popIdx]
      if(pop == "wt"):
         rloc_wt = rlocList[pop]
      else:
         mutString = pop
         rloc_mut = rlocList[pop]

   assert(rloc_wt is not None)
   assert(rloc_mut is not None)

   #true difference rloc matrix
   trueDiff = diffDict[mutString]

   #reconstructed difference rloc matrix
   rloc_wt = reorder_rloc(rloc_wt)
   rloc_mut = reorder_rloc(rloc_mut)
   reconstructedDiff = ((rloc_wt - rloc_mut)**2)**0.5

   #from these final two rlocs calculate RMSE
   trueDiff = reorder_rloc(trueDiff)
   reconstructedDiff = reorder_rloc(reconstructedDiff)

   rmse = calc_rmse(trueDiff, reconstructedDiff)
   return(rmse)

In [19]:
#GET RLOCS FOR THE PERFORMANCE ANALYSIS OF CNR
pickleFile = "/DATA/t.stohn/MRA/scMRA-analysis/notebooks/paperAnalyses/2_CNRAnalysis/bin/SIMULATION_CnrOverMra_NoiseRange_CellNum250_04-19-2023|20:59:39.pickle"
result = pickle.load(open(pickleFile, 'rb'))

headers = result.header
resultFrame = pd.DataFrame(columns=headers)
#add all the existing parameters
resultNumber = len(result.parameter)
for i in range(resultNumber):
    params = result.parameter[i]
    df_length = len(resultFrame)
    resultFrame.loc[df_length] = params
cnrData = resultFrame[resultFrame["MODELTYPE"] == "CNR"]
populationLevelsTmp = cnrData["POPULATIONS"].unique()
print(populationLevelsTmp)

['wt,braf' 'wt,ras']


In [26]:

#get CNR result objects with its rlocs
valueList = ["CNR", 0.5, 250, "wt,ras"]
keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS"]
resultObject, params = result.getResult(keyList, valueList, False) #the result from the simulation

print(len(resultObject))

popStringList = ["wt", "ras"]
diffDict = calculate_diff_dict()
for obj in resultObject:
  print("[   --------------------------------")


  rmse = calculate_rmse_of_network_differences(obj.rloc, popStringList, diffDict)
  print(rmse)
  if(rmse < 0.08):
    print("!!!!!!!!!!")

  #print(obj.rloc)
  print(obj.rloc["wt"] - obj.rloc["ras"])

  print("--------------------------------   ]")



20
[   --------------------------------
0.12147479739357937
             Sos  Ras  Raf1  Mek       Erk  P90Rsk  PI3K  Akt       C3G  Rap1  \
Sos     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Ras     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Raf1    0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Mek     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Erk     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
P90Rsk  0.000000  0.0   0.0  0.0  0.248002     0.0   0.0  0.0  0.000000   0.0   
PI3K    0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Akt     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
C3G     0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
Rap1    0.000000  0.0   0.0  0.0  0.000000     0.0   0.0  0.0  0.000000   0.0   
bRaf   -0.613604  0.0   0.0  0.0  0.000000     0.